# Interval Around the Current Price
This strategy will provide liquidity around the current price in a fixed interval. We will rebalance our position every update interval as discussed in the beginning of chapter 4. Let pc be the current price and a ∈ [10, 40000], then we will set our position to [pc − a, pc + a] at every update interval.
We will fill the interval as much as possible. However, it is usually not possible to use up both tokens, so the remaining token will be left outside the pool.

Import code dependencies

In [11]:
from datetime import timedelta, date

import pandas as pd

from demeter import TokenInfo, Actuator, ChainType, MarketInfo, Strategy, PeriodTrigger, RowData
from demeter.result import performance_metrics
from demeter.uniswap import UniV3Pool, UniLpMarket

Set pandas output format

In [12]:
pd.options.display.max_columns = None
pd.set_option("display.width", 5000)

Custom Intervals Around the Current Price strategy to add liquidity at constant interval around current price.

In [13]:
class IntervalsAroundtheCurrentPriceStrategy(Strategy):
    def __init__(self, a=10, b=1, update_interval=timedelta(days=1)):
        super().__init__()
        self.a = a
        self.b = b

    def initialize(self):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        current_price = lp_market.market_status.data.price

        lp_market.add_liquidity(current_price - self.a, current_price + self.a)
        self.triggers.append(PeriodTrigger(time_delta=timedelta(days=1), do=self.work))

    def work(self, row_data: RowData):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        current_price = row_data.prices[eth.name]
        if len(lp_market.positions) > 0:
            lp_market.remove_all_liquidity()
            lp_market.even_rebalance(row_data.prices[eth.name])

        lp_market.add_liquidity(current_price - self.a, current_price + self.a)

Main logic to run Actuator, init two token and market with key "market1"

In [14]:
usdc = TokenInfo(name="usdc", decimal=6)  # declare  token0
eth = TokenInfo(name="eth", decimal=18)  # declare token1
pool = UniV3Pool(usdc, eth, 0.05, usdc)  # declare pool
market_key = MarketInfo("market1")

actuator = Actuator()  # declare actuator
broker = actuator.broker
market = UniLpMarket(market_key, pool)

broker.add_market(market)
broker.set_balance(usdc, 1800)
broker.set_balance(eth, 1)

actuator.strategy = IntervalsAroundtheCurrentPriceStrategy(400, 200)

market.data_path = "../data"
market.load_data(ChainType.polygon.name, "0x45dda9cb7c25131df268515131f647d726f50608", date(2023, 8, 13), date(2023, 8, 17))
actuator.set_price(market.get_price_from_data())
# actuator.run()

2024-07-22 16:20:54,496 - INFO - start load files from 2023-08-13 to 2023-08-17...
2024-07-22 16:20:54,545 - INFO - load file complete, preparing...
2024-07-22 16:20:54,724 - INFO - data has been prepared


Run actuator with evaluators and save result to files

In [15]:
actuator.run()
print({k: round(v, 5) for k, v in performance_metrics(
    actuator.account_status_df["net_value"], benchmark=actuator.account_status_df["price"]["ETH"]
).items()})

actuator.save_result(
    path="./result",  # save path
    account=True,  # save account status list as a csv file
    actions=True,  # save actions as a json file and a pickle file
)

2024-07-22 16:20:54,748 - INFO - init strategy...
2024-07-22 16:20:54,751 - INFO - start main loop...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 7200/7200 [00:02<00:00, 2792.83it/s]
2024-07-22 16:20:57,334 - INFO - main loop finished
2024-07-22 16:20:57,479 - INFO - Print actuator summary
2024-07-22 16:20:57,666 - INFO - Backtesting finished, execute time 2.9188175201416016s


Final account status                              
Token balance in broker       
USDC      :0                        ETH       :0.19004280               
Position value in markets     
market1(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee(%)    :0.0500                   quote token:USDC                     
positions 
   lower_tick  upper_tick   pending0      pending1        liquidity
0      199327      203830  2.8842556  0.0018592076  360893373781018

Account balance history                           
l1                  net_value tokens              market1                                                                                          price     
l2                              USDC        ETH net_value base_uncollected quote_uncollected base_in_position quote_in_position position_count       ETH USDC
2023-08-13 00:00:00 3648.1245      0 0.20806287 3263.5984    8.1332487e-08                 0       0.79193711              1800     

2024-07-22 16:20:57,779 - INFO - files have saved to ./result\backtest-20240722-162057.account.csv,./result\backtest-20240722-162057.action.json,./result\backtest-20240722-162057.action.pkl


['./result\\backtest-20240722-162057.account.csv',
 './result\\backtest-20240722-162057.action.json',
 './result\\backtest-20240722-162057.action.pkl']